# 🧾 Peppol Readiness Checker - Binder-Friendly Version

In [ ]:
# Module 1: Imports and Constants
import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
from fuzzywuzzy import process
import random

HTML("""
<style>
.widget-label { color: #D04A02 !important; font-weight: bold; }
.output { font-family: 'Segoe UI', sans-serif; }
</style>
""")

## 📄 Upload CSV File

In [ ]:
# Upload CSV manually in Binder
upload = widgets.FileUpload(accept='.csv', multiple=False)
display(upload)

def read_uploaded_file(uploaded):
    if uploaded.value:
        fname = list(uploaded.value.keys())[0]
        content = uploaded.value[fname]['content']
        df = pd.read_csv(io.BytesIO(content))
        df.columns = [col.strip().lower() for col in df.columns]
        return df
    return None

In [ ]:
# Run this cell after uploading to read and display data
df = read_uploaded_file(upload)
if df is not None:
    display(Markdown("### ✅ Data Loaded"))
    display(df.head())
else:
    print("⚠️ Please upload a CSV file to proceed.")

## 🔍 VAT and Peppol Simulated Checks

In [ ]:
# Simulated VAT Check
def simulate_vat_check(vat_number, country):
    if pd.isna(vat_number) or vat_number.strip() == "":
        return "Missing"
    return "Valid" if random.choice([True, False]) else "Invalid"

# Simulated Peppol Check
def simulate_peppol_lookup(name, vat):
    if vat and vat.startswith("VAT"):
        return ("Yes", f"0088:{vat}")
    elif name and len(name) > 3:
        return ("Yes", f"0088:{name[:4].upper()}1234")
    return ("No", "")

In [ ]:
# Run checks on uploaded data
results = []
if df is not None:
    for index, row in df.iterrows():
        name = row.get("name", "")
        vat = row.get("vat", "") or ""
        country = row.get("country", "")

        vat_status = simulate_vat_check(vat, country)
        peppol_status, peppol_id = simulate_peppol_lookup(name, vat if vat_status == "Valid" else "")

        results.append({
            "Name": name,
            "VAT Number": vat,
            "Country": country,
            "VAT Valid": vat_status,
            "Peppol Registered": peppol_status,
            "Peppol ID": peppol_id
        })

    results_df = pd.DataFrame(results)
    display(Markdown("### ✅ Peppol Readiness Check Results"))
    display(results_df)
else:
    print("Upload a file and run the cell again.")

## 🤖 Chatbot Assistant

In [ ]:
chat_input = widgets.Text(placeholder="Ask me anything about Peppol readiness...", layout=widgets.Layout(width='70%'))
chat_output = widgets.Output()

def handle_chat(msg):
    question = msg["new"].lower()
    response = ""
    if "peppol" in question and "id" in question:
        response = "Peppol ID is typically in format '0088:XXXXXXXXXXXX'."
    elif "vat" in question:
        response = "VAT validity is checked via national services or EU's VIES."
    elif "registered" in question:
        response = "A party is Peppol-registered if they have a published SMP entry."
    else:
        response = "I'm here to help! Ask about Peppol, VAT, or registration checks."
    with chat_output:
        print(f"👤 You: {msg['new']}")
        print(f"🤖 Bot: {response}")
    chat_input.value = ""

chat_input.observe(handle_chat, names='value')
display(Markdown("### 🤖 PwC Assistant Chatbot"))
display(chat_input)
display(chat_output)